## BinaryAdroBalaNetTraining

In [28]:
import torchvision.transforms as transforms
from utils.Dataset import Dataset  
from models.classifiers import BinaryAlexNet,MultiAlexNet,BinaryAdroBalaNet
from utils.ModelTrainer import ModelTrainer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt

En aquest fitxer, realitzarem l'entrenament per al classificador binari pre entrenat AlexNet. El primer que realitzarem es l'importació de les imatges que tractarà el model:

In [29]:
oxford=Dataset('species',227)

Extracting and transforming values from train data:  13%|█████▊                                     | 799/5920 [00:03<00:21, 243.72it/s]Corrupt JPEG data: premature end of data segment
Extracting and transforming values from train data:  52%|█████████████████████▊                    | 3074/5920 [00:14<00:17, 161.29it/s]Corrupt JPEG data: 245 extraneous bytes before marker 0xd9
Extracting and transforming values from train data: 100%|██████████████████████████████████████████| 5920/5920 [00:30<00:00, 192.00it/s]
Extracting and transforming values from test data: 100%|███████████████████████████████████████████| 1470/1470 [00:08<00:00, 180.59it/s]

Data loaded succesfully!


Guardem dins dues variables les fotos corresponents al conjunt d'entrenament i de test:

In [30]:
train_loader = torch.utils.data.DataLoader(oxford.train, 64,shuffle= True)
test_loader = torch.utils.data.DataLoader(oxford.test, 100, shuffle= True)

Generam l'iterador amb el qual podrem iterar sobre totes les imatges

In [31]:
iterador =  iter(test_loader)
next(iterador)

[tensor([[[[0.7662, 0.7698, 0.7555,  ..., 0.9969, 0.9935, 0.9825],
           [0.7645, 0.7608, 0.7561,  ..., 0.9832, 0.9940, 0.9876],
           [0.7641, 0.7663, 0.7615,  ..., 0.9725, 0.9954, 0.9925],
           ...,
           [0.3848, 0.3855, 0.3767,  ..., 0.4640, 0.4626, 0.4565],
           [0.3902, 0.3871, 0.3734,  ..., 0.4564, 0.4612, 0.4641],
           [0.3789, 0.3803, 0.3783,  ..., 0.4521, 0.4580, 0.4646]],
 
          [[0.8041, 0.8009, 0.7973,  ..., 0.9922, 0.9953, 0.9970],
           [0.8094, 0.8017, 0.8021,  ..., 0.9844, 0.9947, 0.9972],
           [0.8052, 0.8054, 0.8077,  ..., 0.9821, 0.9938, 0.9945],
           ...,
           [0.4647, 0.4705, 0.4674,  ..., 0.5774, 0.5750, 0.5771],
           [0.4628, 0.4641, 0.4671,  ..., 0.5790, 0.5785, 0.5797],
           [0.4586, 0.4633, 0.4739,  ..., 0.5794, 0.5773, 0.5783]],
 
          [[0.7234, 0.7193, 0.7142,  ..., 0.9903, 0.9945, 0.9920],
           [0.7309, 0.7229, 0.7230,  ..., 0.9784, 0.9941, 0.9964],
           [0.7269, 0.72

Una vegada esta tot preparat, realitzam l'entrenament corresponent al model:

In [ ]:
# Entrenament Binary AdroBalaNet
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
lr = 1e-1

model = BinaryAdroBalaNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

trainer = ModelTrainer(train_loader,test_loader,optimizer,F.cross_entropy,10)

best_model, train_loss, test_loss = trainer.train_test(model, device)

/Users/joanbalaguer/opt/miniconda3/envs/aa2023/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/joanbalaguer/opt/miniconda3/envs/aa2023/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Trainable Parameters:  3649730


  0%|                                                                                                            | 0/10 [00:00<?, ?it/s]

Una vegada finalitzat l'entrenament, generam el fitxer de pesos del model:

In [ ]:
# Guardem els pesos del model
path_to_save = 'pesos/model_weights_Binary_Adro_Bala_Net.pth'
torch.save(best_model.state_dict(), path_to_save)

Finalment, mostrem la gràfica de pèrdua que ha tingut el model durant l'entrenament:

In [ ]:
# Creant el gràfic de pèrdua
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Pèrdua d\'Entrenament')
plt.plot(test_loss, label='Pèrdua de Prova')
plt.title('Funció de Pèrdua Durant l\'Entrenament i la Prova')
plt.xlabel('Èpoques')
plt.ylabel('Pèrdua')
plt.legend()
plt.show()

Una vegada entrenat el model i mostrada la funció de pèrdua, anem a comprovar que realment hem emmagatzemat els pesos del millor model que hem entrenat:

In [ ]:
ruta_modelo = 'weights/model_weights_Binary_Adro_Bala_Net.pth'
model.load_state_dict(torch.load(ruta_modelo))
model.eval()

test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        output = torch.squeeze(output)
        test_loss += F.cross_entropy(output, target, reduction='mean') 
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

accuracy = 100. * correct / len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), accuracy))